In [16]:
import os
import gc
import cv2
import copy
import time
import torch
import random
import string
import joblib
import tifffile
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
from random import randint
from einops import rearrange
from torchvision import models
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from einops.layers.torch import Rearrange
from efficientnet_pytorch import EfficientNet
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split,train_test_split
import warnings; warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold # Sklearn


In [95]:
train_df = pd.read_csv("../input/mayo-clinic-strip-ai/train.csv")
# skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
# for fold,(train_idx, val_idx) in enumerate(skf.split(train_df.image_id, train_df.label)):
#     # train_df.loc[val_idx, 'fold'] = fold
#     print(train_df.loc[train_idx])
#     # print(val_idx)
# train, val = train_test_split(train_df, test_size=0.2, random_state=42, stratify = train_df.label)
# print(train)
# print(val)

train_df.loc[:,'target']=0
idx = train_df.query('label=="LAA"').index
train_df.loc[idx,'target'] = 1
train_df
tiled_img = []
i=0
for arr in train_df.sort_values(['image_id','center_id','target']).values:
    tiled_img.append([arr[0]+f'_{i}', arr[1], arr[2]])

df = pd.DataFrame(columns=['image_id','center_id','target'])
df = pd.concat([df,pd.DataFrame(tiled_img,columns=df.columns)])
df

,image_id,center_id,target
0,006388_0_0,11,006388
1,008e5c_0_0,11,008e5c
2,00c058_0_0,11,00c058
3,01adc5_0_0,11,01adc5
4,026c97_0_0,4,026c97
...,...,...,...
749,fe9645_0_0,3,fe9645
750,fe9bec_0_0,4,fe9bec
751,ff14e0_0_0,6,ff14e0
752,ffec5c_0_0,7,ffec5c


In [56]:
import pandas as pd
df = pd.read_csv('../input/mayo-clinic-strip-ai/sample_submission.csv')
tiled_img = []
for i in range(16):
    for arr in df.sort_values(['patient_id','CE','LAA']).values:
        tiled_img.append([arr[0]+f'_{i}', arr[1], arr[2]])

df_concat = pd.DataFrame(columns=['patient_id','CE','LAA'])
df_concat = pd.concat([df_concat,pd.DataFrame(tiled_img,columns=df.columns)])
df.head()

,patient_id,CE,LAA
0,006388,0.5,0.5
1,008e5c,0.5,0.5
2,00c058,0.5,0.5
3,01adc5,0.5,0.5


In [59]:
df_concat = df_concat.groupby('patient_id').mean()
df_concat

,CE,LAA
patient_id,,
006388_0,0.5,0.5
006388_1,0.5,0.5
006388_10,0.5,0.5
006388_11,0.5,0.5
006388_12,0.5,0.5
...,...,...
01adc5_5,0.5,0.5
01adc5_6,0.5,0.5
01adc5_7,0.5,0.5


In [73]:
import timm
from pprint import pprint
model_names = timm.list_models('*Con*')
pprint(model_names)

['convit_base', 'convit_small', 'convit_tiny']


In [22]:
sub_df = pd.read_csv('../input/mayo-clinic-strip-ai/sample_submission.csv')

# print(sub_df.columns)
sub = pd.DataFrame(columns=sub_df.columns)
for i in [1]:
    single = pd.read_csv(f"submission{i}.csv")
    sub = pd.concat([sub,single])
    # print(sub)
print(sub)

sub = sub.groupby('patient_id').mean()
idx = sub.query('CE>0.5').index
idx2 = sub.query('LAA>0.5').index

sub.loc[idx,'CE'] = 0.6
sub.loc[idx,'LAA'] = 0.4
sub.loc[idx2,'CE'] = 0.4
sub.loc[idx2,'LAA'] = 0.6


print(sub)
idx
#
# sub_df['CE'] = sub['CE'].values
# sub_df['LAA'] = sub['LAA'].values
# print(sub_df)

  patient_id        CE       LAA
0     006388  0.965979  0.034021
1     008e5c  0.950975  0.049025
2     00c058  0.008934  0.991066
3     01adc5  0.979125  0.020875
             CE  LAA
patient_id          
006388      0.6  0.4
008e5c      0.6  0.4
00c058      0.4  0.6
01adc5      0.6  0.4


Index(['006388', '008e5c', '01adc5'], dtype='object', name='patient_id')

下面两个是做tile预测的

In [1]:
import gc
import cv2
import copy
import time
import torch
import random
import string
import joblib
import tifffile
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sns
from random import randint
from einops import rearrange
from torchvision import models
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from einops.layers.torch import Rearrange
from efficientnet_pytorch import EfficientNet
from torch.utils.data import Dataset, DataLoader

test_df = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
tiled_img = []
for i in range(16):
    for arr in test_df.sort_values(['image_id','center_id','patient_id','image_num']).values:
        tiled_img.append([arr[0]+f'_{i}', arr[1], arr[2], arr[3]])

df = pd.DataFrame(columns=['image_id','center_id','patient_id','image_num'])
df = pd.concat([df,pd.DataFrame(tiled_img,columns=df.columns)])


class ImgDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform


    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        paths = "../input/mayo-train-images-size1024-n16/test/"
        try:
            image = cv2.imread(paths + self.df.iloc[index].image_id + ".jpg")
        except:
            image = np.zeros((512,512,3), np.uint8)
        label = 0
        try:
            if len(image.shape) == 5:
                image = image.squeeze().transpose(1, 2, 0)
            if self.transform is not None:
                image = self.transform(image=image)["image"]
        except:
            image = np.zeros((3, 512, 512))
        patient_id = self.df.iloc[index].patient_id
        return image, patient_id


def predict(model, dataloader):
    model.cuda()
    model.eval()
    dataloader = dataloader
    outputs = []
    s = nn.Softmax(dim=1)
    ids = []
    with torch.no_grad():

        for item in tqdm(dataloader, leave=False):
            patient_id = item[1][0]
            ids.append(patient_id)
            print(f'-----------------id:{patient_id}-----------------')
            try:
                images = item[0].cuda().float()
#                 output = model(images)
#                 print(images.shape)
#                 print(images.flip(-1).flip(-2))

                output = model(images)
#                 output = model(images) + model(images.flip(-1)) + model(images.flip(-2)) + model(images.flip(-2).flip(-1))
#                 output = output/4
                print(output)
                outputs.append(s(output.cpu()[:,:2])[0].detach().numpy())
            except:
                outputs.append(s(torch.tensor([[1, 1]]).float())[0].detach().numpy())
                print('!!!!!!!!!!!!!!except occur on predict !!!!!!!!!!!!!!')
        print(outputs)
    return np.array(outputs), ids



In [3]:
import timm
model = torch.jit.load(r'W:\PycharmProjects\kaggle-MC\work\ckpt-tile\coat_fold99_epoch2_0.6651.pth')

#     #
# model = timm.create_model("resnet18",
#                           num_classes=2,
#                           in_chans=3,
#                           pretrained=True)
model.cuda()
model.eval()


batch_size = 1
import albumentations as A
from albumentations.pytorch import ToTensorV2

test_loader = DataLoader(
    ImgDataset(df,
            A.Compose([
                A.Resize(512, 512),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
                ToTensorV2(),
                ])
              ),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

anss, ids = predict(model, test_loader)
prob = pd.DataFrame({"CE" : anss[:,0], "LAA" : anss[:,1], "id" : ids}).groupby("id").mean()
submission = pd.read_csv("../input/mayo-clinic-strip-ai/sample_submission.csv")
submission.CE = prob.CE.to_list()
submission.LAA = prob.LAA.to_list()
submission.to_csv(f"submission{i}.csv", index = False)
i +=1

  0%|          | 0/64 [00:00<?, ?it/s]

-----------------id:006388-----------------
tensor([[ 0.0752, -0.1442]], device='cuda:0')
-----------------id:008e5c-----------------
tensor([[ 0.8084, -0.9188]], device='cuda:0')
-----------------id:00c058-----------------
tensor([[ 2.9419, -3.5244]], device='cuda:0')
-----------------id:01adc5-----------------
tensor([[ 0.9811, -1.1265]], device='cuda:0')
-----------------id:006388-----------------
tensor([[ 0.2937, -0.3328]], device='cuda:0')
-----------------id:008e5c-----------------
tensor([[ 1.0819, -1.2279]], device='cuda:0')
-----------------id:00c058-----------------
tensor([[ 1.7812, -2.1214]], device='cuda:0')
-----------------id:01adc5-----------------
tensor([[ 0.7894, -0.8970]], device='cuda:0')
-----------------id:006388-----------------
tensor([[ 0.4229, -0.4731]], device='cuda:0')
-----------------id:008e5c-----------------
tensor([[ 0.6325, -0.7186]], device='cuda:0')
-----------------id:00c058-----------------
tensor([[ 2.3684, -2.8955]], device='cuda:0')
----------

In [11]:
torch.cuda.empty_cache

<function torch.cuda.memory.empty_cache() -> None>

In [5]:
import cv2
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
generate_new = False
test_df = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
class ImgDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.train = 'label' in df.columns
    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        if(generate_new):
            paths = ["../test/", "../train/"]
        else:
            paths = ["../input/jpg-images-strip-ai/test/", "../input/jpg-images-strip-ai/train/"]
        try:
            image = cv2.imread(paths[self.train] + self.df.iloc[index].image_id + ".jpg")
        except:
            image = np.zeros((512,512,3), np.uint8)
        label = 0
        try:
            if len(image.shape) == 5:
                image = image.squeeze().transpose(1, 2, 0)
            image = cv2.resize(image, (512, 512)).transpose(2, 0, 1)
        except:
            image = np.zeros((3, 512, 512))
        if(self.train):
            label = {"CE" : 0, "LAA": 1}[self.df.iloc[index].label]
        patient_id = self.df.iloc[index].patient_id
        return image, label, patient_id
def predict(model, dataloader):
    model.cuda()
    model.eval()
    dataloader = dataloader
    outputs = []
    s = nn.Softmax(dim=1)
    ids = []
    for item in tqdm(dataloader, leave=False):
        patient_id = item[2][0]
        try:
            images = item[0].cuda().float()
            ids.append(patient_id)
            output = model(images) + model(images.flip(-1)) + model(images.flip(-2)) + model(images.flip(-2).flip(-1))
            output = output/4
            outputs.append(s(output.cpu()[:,:2])[0].detach().numpy())
        except:
            ids.append(patient_id)
            outputs.append(s(torch.tensor([[1, 1]]).float())[0].detach().numpy())
    return np.array(outputs), ids
model = torch.jit.load('./ckpt/finetune_fold0.pth')
# model = torch.jit.load('./ckpt/model_ori.pth')
batch_size = 1
test_loader = DataLoader(
    ImgDataset(test_df),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0
)

anss, ids = predict(model, test_loader)
prob = pd.DataFrame({"CE" : anss[:,0], "LAA" : anss[:,1], "id" : ids}).groupby("id").mean()
prob

,CE,LAA
id,,
006388,0.609391,0.390609
008e5c,0.548586,0.451414
00c058,0.592479,0.407522
01adc5,0.414521,0.585479
